<a href="https://colab.research.google.com/github/brandontan99/Self_Love_App/blob/master/OneVsRest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/brandontan99/Self_Love_App.git
%cd Self_Love_App

Cloning into 'Self_Love_App'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 149 (delta 81), reused 39 (delta 15), pack-reused 0
Receiving objects: 100% (149/149), 1.80 MiB | 12.46 MiB/s, done.
Resolving deltas: 100% (81/81), done.
/content/Self_Love_App


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from data_cleaning import *
from Data_Normalization import *
import time
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
import sklearn
import statistics
import joblib
from copy import deepcopy

In [3]:
df=pd.read_csv("WID3006 ML Questionnaire.csv")
df=data_cleaning(df)
from Data_Normalization import data_encoding, data_normalization
df = data_encoding(df)
df_norm = data_normalization(df)
df_norm

,Gender: _Male,What is your current occupation?_Retired,What is your current occupation?_Unemployed,What is your current occupation?_University student,What boosts your confidence ? _By leading others to success,What boosts your confidence ? _Get the most/ special attention among the members,What boosts your confidence ? _When someone acknowledges you,What boosts your confidence ? _When you accomplish a project,I prefer to spend my money on...._Food,I prefer to spend my money on...._Home Improvements,I prefer to spend my money on...._The latest fashion,I prefer to spend my money on...._The latest technology,How do you organize your thoughts? Please pick whichever is closest._I make a chart or graph,How do you organize your thoughts? Please pick whichever is closest._I talk to myself out loud,How do you organize your thoughts? Please pick whichever is closest._I write it down in my calendar,How do you organize your thoughts? Please pick whichever is closest._Jotting it down on a scrap of paper,Choose a pet which you prefer to keep._Cat,Choose a pet which you prefer to keep._Dog,Choose a pet which you prefer to keep._Fish,Choose a pet which you prefer to keep._Hamster,Choose a pet which you prefer to keep._Horse,Choose a pet which you prefer to keep._I'm not a pet person,Choose a pet which you prefer to keep._Rabbit,Choose a pet which you prefer to keep._Snake,Choose a pet which you prefer to keep._Tortoise,What is your favorite time of the day?_Evening,What is your favorite time of the day?_Morning,What is your favorite time of the day?_Night,Would you rather visit the future or the past?_The future,Would you rather visit the future or the past?_The past,What do you worry more about the most?_Money,What do you worry more about the most?_The state of the world,What do you worry more about the most?_Your family and friends,What do you worry more about the most?_Your future,"When you retire, you'd like to live..._Exactly where I live now","When you retire, you'd like to live..._In a hectic big city","When you retire, you'd like to live..._In a small town","When you retire, you'd like to live..._Overseas","When you retire, you'd like to live..._Traveling the world",What is your favorite color?_Blue,What is your favorite color?_Green,What is your favorite color?_Purple,What is your favorite color?_Red,What is your favorite color?_White,What is your favorite color?_Yellow,What is your learning style? (Pick one that benefit you the most)_By attending online courses,What is your learning style? (Pick one that benefit you the most)_By doing assignments,What is your learning style? (Pick one that benefit you the most)_By doing tutorial/lab questions,What is your learning style? (Pick one that benefit you the most)_By reading a physical book,What is your learning style? (Pick one that benefit you the most)_By reading an e-Book,Do you enjoy socializing with large groups of people?,Do you enjoy challenges?,How creative of a person do you think you are?,How logical of a person do you think you are?,Would you prefer to engage your brain more than your body?,Are you a curious person?,Are you a perfectionist?,Are you a trusting person?,Do you have lot of patience?,Do you organize your schedule well?,Do you like to sit in front of a computer for long hours?,Do you enjoy making others happy?,Can you understand others' perspectives and feelings?,How confident are you in your own abilities?,Sports and Outdoors,Games,Spiritual and Mental,Performing Arts,Arts and Craft,Food and Drinks,Collecting,Rejuvenation
0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0.25,0.50,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,1.00,0.75,0.50,0.50,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0.75,0.75,0.75,0.75,0.50,1.00,1.00,0.50,0.75,0.75,0.75,1.00,0.75,1.00,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,

In [26]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import train_test_split
from feature_selection import *
df = pd.read_csv("WID3006 ML Questionnaire.csv")
df = data_cleaning(df)
    # label encoding is used instead of one-hot encoding because one-hot encoding will create many features for one question
    # but label encoding is able to perform chi-square test between a single question as a whole and the hobby  
x_df, y_df = label_encoding(df)
    # only evaluate on train data to prevent data leakage
x_train_label_encoded, _, y_train, _ = train_test_split(x_df.to_numpy(), y_df.to_numpy(), test_size=0.2, random_state=1)
chi2_result = chi2_analysis(x_train_label_encoded, x_df, y_train)

    # to pick the selected best questions as features
    # Eg: choose the best 13 features as below
df = pd.read_csv("WID3006 ML Questionnaire.csv")
df = data_cleaning(df)
df = data_encoding(df)
df_norm = data_normalization(df)
best_k_features = select_best_k_features(chi2_result, k=13)
x = filter_features(best_k_features, df_norm)
y=df_norm.iloc[:,-8:]
y.sum(axis=1).describe()

count    192.000000
mean       2.802083
std        1.250087
min        1.000000
25%        2.000000
50%        3.000000
75%        4.000000
max        7.000000
dtype: float64

In [4]:

# Using pipeline for applying logistic regression and one vs rest classifier
OneVsRestClassifier.multilabel_=True
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
NB_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
SVC_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])

In [77]:
def trainModel(pipeline,x,y):
  f1_list=[]
  pipelineList=[]
  x_train, x_test,y_train,y_test = train_test_split(x,y, random_state=0, test_size=0.25, shuffle=True)
  x_test=x_test.iloc[20:,:]
  y_test=y_test.iloc[20:,:]
  for i,hobby in enumerate(y.columns):
      # Training logistic regression model on train data
      pipelineList.append(deepcopy(pipeline).fit(x_train, y_train[hobby]))
    
      # calculating test accuracy
      prediction = pipelineList[i].predict(x_test)
      print(prediction)
      f1=sklearn.metrics.f1_score(y_test[hobby], prediction,average="micro")
      f1_list.append(f1)
      print(f1)
  print('Average f1-score: {}% ({})'.format(statistics.mean(f1_list)*100,statistics.stdev(f1_list)*100))
  return pipelineList

def predict(pipelineList,x_pred):
  x_pred=np.array(x_pred).reshape(-1,x.shape[1])
  pred=[]
  for i,hobby in enumerate(y.columns):
    prediction = pipelineList[i].predict(x_pred)
    if prediction[0]==1:
      pred.append(hobby)
  print(pred)

pipeline=LogReg_pipeline
pipelineList=trainModel(pipeline,x,y)
for i in range(50): 
  predict(pipelineList,x.iloc[i,:])

[0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0.
 0. 1. 1. 1.]
0.5357142857142857
[0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1.
 0. 0. 0. 1.]
0.6428571428571429
[1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0.
 1. 1. 0. 1.]
0.5
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0.
 1. 0. 1. 1.]
0.5714285714285714
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 0.]
0.7857142857142857
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
0.5714285714285714
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
0.8928571428571429
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
0.8571428571428571
Average f1-score: 66.96428571428571% (15.361295119718976)
['Sports and Outdoors', 'Games', 'Spiritual and Mental']
['Sports and Outdoors', 'Games', 'Spiritual and Ment

In [ ]:
pipeline=NB_pipeline
dict_acc={}
f1_list=[]
for i in range(1):
  train, test = train_test_split(df_norm, random_state=1, test_size=0.25, shuffle=True)

  x_train = train.iloc[:, :76]
  y_train = train.iloc[:, 76:]

  x_test = test.iloc[:, :76]
  y_test = test.iloc[:, 76:]
  for hobby in df_norm.iloc[:,76:].columns:
        #print('**Processing {} hobbies...**'.format(hobby))
    
        # Training logistic regression model on train data
      pipeline.fit(x_train, train[hobby])
    
        # calculating test accuracy
      prediction = pipeline.predict(x_test)
      f1=sklearn.metrics.f1_score(test[hobby], prediction,average="micro")
        
        #dict_acc[hobby]=accuracy_score(test[hobby], prediction)
        #print('Test accuracy is {}'.format(accuracy_score(test[hobby], prediction)))
        #print("\n")
      f1_list.append(f1)
    #print('Test accuracy is {}'.format(acc/len(df_norm.iloc[:,24:].columns)))
print('Average accuracy: {}% ({})'.format(statistics.mean(acc_list)*100,statistics.stdev(acc_list)*100))
    #sort_acc = sorted(dict_acc.items(), key=lambda x: x[1], reverse=True)
    #for i in sort_acc:
	    #print(i[0], i[1])
log_model=pipeline
tp=0
fp=0
fn=0
for i in range(1): 
  train, test = train_test_split(df_norm, random_state=1, test_size=0.25, shuffle=True)
  for j in range(len(test)):
    x_test = test.iloc[j, :76][None,:]
    y_test = test.iloc[j, 76:]
    #prediction(x_test,y_test,log_model)
    count=0
    for hobby in df_norm.iloc[:,76:].columns:
        pipeline.fit(x_train, train[hobby])
        prediction = pipeline.predict(x_test)
        if prediction==y_test[count]:
          if prediction==1:
            tp+=1
        else:
          if prediction==1:
            fp+=1
          else:
            fn+=1
        count+=1
precision=tp/(tp+fp)
recall=tp/(tp+fn)
print("F1: ",2*precision*recall/(precision+recall))

Average accuracy: 84.16512059369202% (0.8779189936121612)
F1:  0.362962962962963


In [ ]:
import warnings
warnings.filterwarnings("ignore")
pipeline=SVC_pipeline
dict_acc={}
f1_list=[]
for i in range(1):
  train, test = train_test_split(df_norm, random_state=i, test_size=0.25, shuffle=True)

  x_train = train.iloc[:, :76]
  y_train = train.iloc[:, 76:]

  x_test = test.iloc[:, :76]
  y_test = test.iloc[:, 76:]
  for hobby in df_norm.iloc[:,76:].columns:
        #print('**Processing {} hobbies...**'.format(hobby))
    
        # Training logistic regression model on train data
      pipeline.fit(x_train, train[hobby])
    
        # calculating test accuracy
      prediction = pipeline.predict(x_test)
      f1=sklearn.metrics.f1_score(test[hobby], prediction,average="micro")
        
        #dict_acc[hobby]=accuracy_score(test[hobby], prediction)
        #print('Test accuracy is {}'.format(accuracy_score(test[hobby], prediction)))
        #print("\n")
      f1_list.append(f1)
    #print('Test accuracy is {}'.format(acc/len(df_norm.iloc[:,24:].columns)))
print('Average accuracy: {}% ({})'.format(statistics.mean(acc_list)*100,statistics.stdev(acc_list)*100))
    #sort_acc = sorted(dict_acc.items(), key=lambda x: x[1], reverse=True)
    #for i in sort_acc:
	    #print(i[0], i[1])
log_model=pipeline
tp=0
fp=0
fn=0
for i in range(1): 
  train, test = train_test_split(df_norm, random_state=i, test_size=0.25, shuffle=True)
  for j in range(10,20):
    x_test = test.iloc[j, :76][None,:]
    y_test = test.iloc[j, 76:]
    #prediction(x_test,y_test,log_model)
    count=0
    for hobby in df_norm.iloc[:,76:].columns:
        pipeline.fit(x_train, train[hobby])
        prediction = pipeline.predict(x_test)
        if prediction==y_test[count]:
          if prediction==1:
            tp+=1
        else:
          if prediction==1:
            fp+=1
          else:
            fn+=1
        print("{}: ".format(hobby),prediction," ",y_test[count])
        count+=1
precision=tp/(tp+fp)
recall=tp/(tp+fn)
print("F1: ",2*precision*recall/(precision+recall))

Average accuracy: 84.16512059369202% (0.8779189936121612)
Team sports:  [1.]   0.0
Badminton:  [1.]   1.0
Dancing:  [0.]   0.0
Fishing:  [1.]   0.0
Playing computer games:  [0.]   1.0
Playing board games:  [0.]   0.0
Reading:  [0.]   1.0
Exercising:  [1.]   0.0
Writing:  [0.]   0.0
Gardening:  [0.]   0.0
Painting:  [0.]   0.0
Cooking:  [0.]   0.0
Singing:  [0.]   0.0
Puzzles:  [0.]   0.0
Collecting:  [0.]   0.0
Watching movies:  [0.]   0.0
Watching TV series:  [0.]   0.0
Sleeping:  [0.]   0.0
Photography and Videography:  [0.]   0.0
Listening to music:  [0.]   0.0
Crocheting:  [0.]   0.0
Playing a musical instrument:  [0.]   0.0
Team sports:  [0.]   1.0
Badminton:  [0.]   1.0
Dancing:  [1.]   0.0
Fishing:  [0.]   0.0
Playing computer games:  [0.]   0.0
Playing board games:  [0.]   0.0
Reading:  [0.]   1.0
Exercising:  [0.]   1.0
Writing:  [0.]   1.0
Gardening:  [0.]   0.0
Painting:  [0.]   0.0
Cooking:  [0.]   0.0
Singing:  [0.]   0.0
Puzzles:  [0.]   1.0
Collecting:  [0.]   0.0
Watchi

In [ ]:
def prediction(df,df_ans,pipeline):
    x_train = df_norm.iloc[:, :76]
    count=0
    for hobby in df_norm.iloc[:,76:].columns:
        pipeline.fit(x_train, df_norm[hobby])
        prediction = pipeline.predict(df)
        print("{}: ".format(hobby),prediction," ",df_ans[count])
        count+=1
    print()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
for i in range(1): 
  train, test = train_test_split(df_norm, random_state=i, test_size=0.25, shuffle=True)
  print(test.shape)
  for j in range(10,20):
    x_test = test.iloc[j, :76][None,:]
    y_test = test.iloc[j, 76:]
    prediction(x_test,y_test,nb_model)

(49, 98)
Team sports:  [0.]   0.0
Badminton:  [1.]   1.0
Dancing:  [0.]   0.0
Fishing:  [0.]   0.0
Playing computer games:  [1.]   1.0
Playing board games:  [0.]   0.0
Reading:  [1.]   1.0
Exercising:  [1.]   0.0
Writing:  [0.]   0.0
Gardening:  [0.]   0.0
Painting:  [0.]   0.0
Cooking:  [0.]   0.0
Singing:  [0.]   0.0
Puzzles:  [0.]   0.0
Collecting:  [0.]   0.0
Watching movies:  [0.]   0.0
Watching TV series:  [0.]   0.0
Sleeping:  [0.]   0.0
Photography and Videography:  [0.]   0.0
Listening to music:  [0.]   0.0
Crocheting:  [0.]   0.0
Playing a musical instrument:  [0.]   0.0

Team sports:  [0.]   1.0
Badminton:  [0.]   1.0
Dancing:  [0.]   0.0
Fishing:  [0.]   0.0
Playing computer games:  [0.]   0.0
Playing board games:  [0.]   0.0
Reading:  [1.]   1.0
Exercising:  [0.]   1.0
Writing:  [0.]   1.0
Gardening:  [0.]   0.0
Painting:  [0.]   0.0
Cooking:  [0.]   0.0
Singing:  [0.]   0.0
Puzzles:  [0.]   1.0
Collecting:  [0.]   0.0
Watching movies:  [0.]   0.0
Watching TV series:  [0.]

In [ ]:
acc=[]
for i in range(10): 
  train, test = train_test_split(df_norm, random_state=i, test_size=0.25, shuffle=True)
  for j in range(len(test)):
    x_test = test.iloc[j, :24][None,:]
    y_test = test.iloc[j, 24:]
    acc.append(prediction(x_test,y_test,svc_model))
print(statistics.mean(acc))

0.875139146567718
